**SESRec with Tensorflow**

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import precision_score, recall_score, f1_score

# Loading the train and test datasets
train_file = 'dataset/train_inter.tsv'
test_file = 'dataset/test_inter.tsv'

# Loading data from corresponding TSV files
train_data = pd.read_csv(train_file, sep='\t')
test_data = pd.read_csv(test_file, sep='\t')

# Data preprocessing and mapping to integer indices for metadata
train_data = train_data.dropna(subset=['rec_his', 'src_his', 'ts', 'label'])
test_data = test_data.dropna(subset=['rec_his', 'src_his', 'ts', 'label'])

X_train = train_data[['rec_his', 'src_his', 'ts']].values
y_train = train_data['label'].values

X_test = test_data[['rec_his', 'src_his', 'ts']].values
y_test = test_data['label'].values

# Building the recommender model with class weights and L2 regularization
model = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),  # Define input shape here
    keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(1, activation='sigmoid')
])


# Configuring the optimizer with a learning rate scheduler
initial_learning_rate = 0.1
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100, decay_rate=0.9, staircase=True
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, clipvalue=1.0)

# Compiling the model with binary cross-entropy loss and class weights
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Implementing early stopping
early_stopping = keras.callbacks.EarlyStopping(patience=6, restore_best_weights=True)

# Training the model with class weights
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Evaluating the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Binary crossentropy: {loss}")
print(f"Test Accuracy: {accuracy}")

# Making predictions on the test set
y_pred = model.predict(X_test)
integer_predictions = [1 if prediction >= 0.5 else 0 for prediction in y_pred]

# Calculating precision, recall, and F1 score
precision = precision_score(y_test, integer_predictions)
recall = recall_score(y_test, integer_predictions)
f1 = f1_score(y_test, integer_predictions)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# Recommendations for a user as per the user history rec_his
user = X_test[0:1]
predictions = model.predict(user)
user_integer_predictions = [1 if prediction >= 0.5 else 0 for prediction in predictions]
print(f"Recommendations for the user: {user_integer_predictions}")